# 실습 세션 3: 여행 플래너 Multi-Agent 앱의 Docker 빌드 및 AKS 배포

이 세션에서는 세션 2에서 만든 여행 플래너 멀티에이전트 앱을 Docker 이미지로 빌드하고, Azure Kubernetes Service(AKS)에 배포하는 핵심 실습만 다룹니다.

- **Docker**: 애플리케이션과 그 실행 환경(라이브러리, 설정 등)을 하나의 컨테이너 이미지로 패키징하는 기술입니다. 컨테이너는 어디서나 동일하게 실행되므로, 개발-테스트-운영 환경 간 일관성을 보장합니다.
- **Kubernetes**: 컨테이너화된 애플리케이션의 배포, 확장, 관리를 자동화하는 오픈소스 플랫폼입니다. 여러 대의 서버(노드)에서 컨테이너를 효율적으로 운영할 수 있습니다.
- **AKS(Azure Kubernetes Service)**: Microsoft Azure에서 제공하는 완전관리형 Kubernetes 서비스입니다. 인프라 관리 부담 없이 손쉽게 Kubernetes 클러스터를 생성·운영할 수 있습니다.

이 실습에서는 Docker로 앱을 컨테이너화하고, AKS에 배포하여 실제 클라우드 환경에서 운영하는 과정을 단계별로 경험합니다.

## Docker, Kubernetes, AKS 아키텍처 및 주요 컴포넌트

### Docker에서 사용자가 할 수 있는 주요 작업
- **Dockerfile 작성**: 앱 실행 환경과 빌드 과정을 정의하는 파일을 작성합니다.
- **이미지 빌드**: `docker build` 명령어로 Docker 이미지를 생성합니다.
- **로컬 테스트**: `docker run`으로 이미지를 로컬에서 실행해 정상 동작을 확인합니다.
- **이미지 태깅**: `docker tag`로 이미지를 ACR에 업로드할 이름으로 태깅합니다.
- **ACR 로그인**: `az acr login` 명령어로 Azure Container Registry에 인증합니다.
- **이미지 푸시**: `docker push`로 이미지를 ACR에 업로드합니다.

### Kubernetes & AKS 아키텍처 다이어그램
```
[사용자]
   |
   v
[kubectl, Azure CLI 등]
   |
   v
+-----------------------------+
|      Control Plane (AKS)     |  <-- Microsoft가 완전 관리
|-----------------------------|
|  - API Server               |
|  - Scheduler                |
|  - Controller Manager       |
|  - etcd                     |
+-----------------------------+
           |
           v
+-----------------------------+
|         Node Pool           |
|-----------------------------|
|  [Worker Node 1]            |
|   - Kubelet                 |
|   - Kube Proxy              |
|   - Pod(s)                  |
|     - Container(s)          |
|-----------------------------|
|  [Worker Node 2]            |
|   - Kubelet                 |
|   - Kube Proxy              |
|   - Pod(s)                  |
|     - Container(s)          |
|-----------------------------|
|  ...                        |
+-----------------------------+
           |
           v
[Azure Container Registry(ACR)]
```
- 사용자는 kubectl, Azure CLI 등으로 클러스터에 명령을 전달합니다.
- Control Plane(마스터 노드)은 AKS에서 Microsoft가 완전 관리하며, 클러스터의 상태와 스케줄링을 담당합니다.
- Node Pool에는 여러 Worker Node가 포함되어 실제 컨테이너(Pod)를 실행합니다.
- 각 Worker Node는 Kubelet, Kube Proxy, 여러 Pod(컨테이너)로 구성됩니다.
- 컨테이너 이미지는 Azure Container Registry(ACR) 등에서 가져와 실행할 수 있습니다.

### 주요 컴포넌트 설명 (Kubernetes/AKS)
- **Azure Container Registry(ACR)**: Azure에서 제공하는 프라이빗 Docker 이미지 저장소. 이미지를 안전하게 저장·관리·배포.
- **AKS 클러스터**: Azure에서 관리하는 Kubernetes 클러스터. 여러 노드(서버)로 구성되어 컨테이너화된 앱을 배포·운영.

#### Master Node (Control Plane, AKS에서 Microsoft가 완전 관리)
- **API Server**: 클러스터의 모든 요청을 받아들이는 프론트엔드. kubectl, 대시보드 등 외부와의 통신을 담당합니다.
- **Scheduler**: 새로 생성된 파드를 어떤 노드에 할당할지 결정합니다.
- **Controller Manager**: 클러스터의 상태를 지속적으로 모니터링하고, 원하는 상태(desired state)로 유지되도록 제어합니다. (예: 파드 복제, 노드 상태 감시 등)
- **etcd**: 클러스터의 모든 설정과 상태 정보를 저장하는 분산 Key-Value 저장소입니다.

#### Worker Node (사용자 관리)
- **Kubelet**: 각 워커 노드에서 파드와 컨테이너의 상태를 관리하고, 컨트롤 플레인과 통신합니다.
- **Kube Proxy**: 네트워크 프록시 및 로드 밸런서 역할을 하여, 파드 간/외부와의 네트워크 트래픽을 관리합니다.
- **Pod**: Kubernetes에서 배포·관리되는 최소 단위. 하나 이상의 컨테이너(containerd 등)로 구성됩니다.
- **Container**: 실제 애플리케이션이 실행되는 환경. containerd 등으로 구동됩니다.

이 구조를 통해 개발자는 코드를 Docker 이미지로 패키징하고, ACR에 저장한 뒤, AKS 클러스터에 배포하여 대규모 서비스 운영이 가능합니다.

## 1. Docker CLI 설치 확인 (GitHub Codespaces 환경)

GitHub Codespaces 환경에는 이미 Docker CLI가 설치되어 있으므로 별도의 설치가 필요하지 않습니다.

- Codespaces 터미널에서 아래 명령어로 Docker CLI가 정상적으로 설치되어 있는지 확인하세요.

```sh
docker --version
```

- 위 명령어 실행 시 Docker 버전 정보가 출력되면 정상입니다.
- 로컬 환경에서 실습할 경우 [Docker Desktop](https://www.docker.com/products/docker-desktop/)을 별도로 설치해야 합니다.

## 2. Dockerfile 작성

아래는 여행 플래너 앱을 컨테이너화하기 위한 Dockerfile 예시입니다.

- Dockerfile은 컨테이너의 실행 환경, 의존성, 복사할 파일, 실행 명령어 등을 정의하는 핵심 파일입니다.
- 실습에서는 Python 기반 앱을 위한 베이스 이미지, 의존성 설치, 소스 복사, 엔트리포인트 지정 등 기본적인 Dockerfile 구조를 익힐 수 있습니다.
- Dockerfile을 수정하면 다양한 환경(예: 추가 패키지, 환경 변수, 빌드 단계 등)에 맞게 이미지를 커스터마이즈할 수 있습니다.
- 작성한 Dockerfile은 이후 단계에서 이미지 빌드 및 배포에 활용됩니다.

### 주요 Dockerfile 키워드 설명
- **FROM**: 베이스 이미지를 지정합니다. (예: python:3.12-slim)
- **WORKDIR**: 컨테이너 내 작업 디렉터리를 설정합니다.
- **COPY**: 파일 또는 디렉터리를 컨테이너 이미지로 복사합니다.
- **RUN**: 컨테이너 빌드 시 실행할 명령어를 지정합니다. (예: 패키지 설치)
- **ENTRYPOINT**: 컨테이너가 시작될 때 실행할 기본 명령어를 지정합니다.

아래 셀의 예시 Dockerfile을 참고하여, 필요에 따라 키워드와 명령어를 추가·수정해보세요.

In [ ]:
%%writefile Dockerfile
# Step 1: Base image with Python environment
FROM python:3.12-slim AS base

# Step 2: Set working directory
WORKDIR /app

# Step 3: Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Step 4: Copy the project files into the container
COPY . .

# Step 5: Define the entry point
ENTRYPOINT ["python", "/app/travel_planning_agent.py"]

## 3. Docker 이미지 빌드

이 단계에서는 작성한 Dockerfile을 기반으로 실제 컨테이너 이미지를 빌드합니다.

- `docker build` 명령어를 사용해 소스코드와 Dockerfile이 위치한 디렉터리에서 이미지를 생성합니다.
- 빌드된 이미지는 로컬 환경에서 테스트하거나, 이후 Azure Container Registry(ACR)로 푸시할 수 있습니다.
- 이미지 태그(`-t` 옵션)를 활용해 버전 관리 및 ACR 업로드용 이름을 지정할 수 있습니다.
- 빌드 과정에서 오류가 발생하면 Dockerfile의 경로, 의존성, COPY/RUN 명령어 등을 점검하세요.
- 빌드가 완료되면 `docker images` 명령어로 생성된 이미지를 확인할 수 있습니다.

In [ ]:
# Docker 이미지 빌드
!docker build -t travel-planning-agent-app:v1 .

## 4. Azure CLI 설치 (GitHub Codespaces 환경)

이 단계에서는 Azure 리소스(AKS, ACR 등)를 생성하고 관리하기 위해 필요한 Azure CLI(az)를 설치합니다.

- Azure CLI는 Azure 리소스의 생성, 관리, 배포, 모니터링 등 다양한 작업을 명령줄에서 수행할 수 있는 도구입니다.
- GitHub Codespaces 환경에는 기본적으로 Azure CLI가 설치되어 있지 않을 수 있으므로, 아래 명령어로 설치를 진행합니다.
- 설치가 완료되면 `az --version` 명령어로 정상적으로 설치되었는지 확인하세요.
- 이후 단계에서는 az 명령어를 사용해 리소스 그룹, 컨테이너 레지스트리, AKS 클러스터 등 다양한 Azure 리소스를 생성하고 관리할 수 있습니다.
- 로컬 환경에서 실습할 경우에도 동일하게 Azure CLI를 설치해야 하며, 공식 문서의 설치 가이드를 참고할 수 있습니다.

```
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
```

## 5. AKS 클러스터 및 ACR 레지스트리 생성

이 단계에서는 Azure에서 컨테이너 레지스트리(ACR)와 AKS 클러스터를 생성합니다.

- AKS(Azure Kubernetes Service)는 완전관리형 Kubernetes 클러스터로, 인프라 관리 부담 없이 컨테이너 앱을 배포·운영할 수 있습니다.
- ACR(Azure Container Registry)는 Docker 이미지를 저장·관리·배포할 수 있는 Azure의 프라이빗 레지스트리입니다.
- 아래 명령어를 통해 리소스 그룹, ACR, AKS 클러스터를 순차적으로 생성합니다.
- 클러스터 생성 시 --attach-acr 옵션을 사용하면, AKS에서 ACR 이미지를 바로 사용할 수 있도록 권한이 자동으로 연결됩니다.
- 각 명령어 실행 후 결과 메시지를 확인하고, 오류 발생 시 리소스 이름 중복, 권한, 지역(location) 설정 등을 점검하세요.
- 클러스터 인증 정보를 가져오면 kubectl로 클러스터에 바로 연결할 수 있습니다.

In [ ]:
# Azure 계정 로그인
!az login

In [ ]:
# Azure 리소스 그룹 생성 (예시)
!az group create --name myResourceGroup --location koreacentral

In [ ]:
# Azure Container Registry 생성 (예시)
!az acr create --resource-group myResourceGroup --name <ACR-REGISTRY-NAME> --sku Basic

In [ ]:
# AKS 클러스터 생성 (예시)
!az aks create --resource-group myResourceGroup --name myAKSCluster --node-count 1 --enable-addons monitoring --generate-ssh-keys --attach-acr <ACR-REGISTRY-NAME>

In [ ]:
# AKS 클러스터 인증 정보 가져오기 및 연결
!az aks get-credentials --resource-group myResourceGroup --name myAKSCluster

In [ ]:
# 연결된 AKS 클러스터 확인 (kubectl)
!kubectl get nodes

이제 위에서 생성한 ACR과 AKS를 활용해 이미지를 푸시하고, 클러스터에 배포할 수 있습니다.

## 6. Kubernetes 배포를 위한 YAML 파일 작성

이 단계에서는 AKS에 애플리케이션을 배포하기 위한 Kubernetes 배포 설정 파일(deployment.yaml)을 작성합니다.

- YAML 파일은 배포할 컨테이너 이미지, 파드 수(replicas), 리소스, 네트워크, 환경 변수 등 다양한 배포 옵션을 선언적으로 정의할 수 있습니다.
- deployment.yaml에는 앱 이름, 사용할 이미지, 파드 수, 레이블, 컨테이너 실행 명령어 등을 명시합니다.
- 실제 배포 시에는 이미지 경로(`image:`)를 본인의 ACR 주소로 교체해야 하며, 필요에 따라 환경 변수, 볼륨, 포트 등도 추가할 수 있습니다.
- YAML 파일을 활용하면 동일한 설정으로 여러 환경(개발, 테스트, 운영)에 일관되게 배포할 수 있습니다.
- 작성한 YAML 파일은 이후 kubectl apply 명령어로 클러스터에 배포할 때 사용됩니다.

In [ ]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: travel-planning-agent-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: travel-planning-agent-app
  template:
    metadata:
      labels:
        app: travel-planning-agent-app
    spec:
      containers:
      - name: travel-planning-agent-app
        image: {{ACR-REGISTRY-NAME}}/travel-planning-agent-app:v1  # 실제 ACR 경로로 교체

## 7. AKS에 배포 (kubectl)

이 단계에서는 빌드한 이미지를 Azure Container Registry(ACR)에 푸시하고, 작성한 YAML 파일을 이용해 AKS 클러스터에 애플리케이션을 배포합니다.

- 먼저 이미지를 ACR에 업로드(`docker push`)하고, AKS에서 해당 이미지를 사용할 수 있도록 합니다.
- `kubectl apply -f deployment.yaml` 명령어로 YAML 파일의 설정에 따라 파드, 디플로이먼트 등 리소스가 생성됩니다.
- 배포 후 `kubectl get pods`, `kubectl get deployments` 등으로 리소스 상태를 확인할 수 있습니다.
- 배포 과정에서 이미지 경로, 네트워크 설정, 리소스 한계 등 오류가 발생할 수 있으니, 로그와 상태를 꼼꼼히 점검하세요.
- 필요에 따라 서비스(Service), 인그레스(Ingress) 등 추가 리소스를 정의해 외부 접근을 설정할 수 있습니다.

In [ ]:
# 1. ACR에 이미지 푸시 (예시)
# !az acr login --name <ACR-REGISTRY-NAME>
# !docker tag travel-planning-agent-app:v1 {{ACR-REGISTRY-NAME}}/travel-planning-agent-app:v1
# !docker push {{ACR-REGISTRY-NAME}}/travel-planning-agent-app:v1

# 2. AKS에 배포
# !kubectl apply -f deployment.yaml

---

- `{{ACR-REGISTRY-NAME}}`는 실제 Azure Container Registry 주소로 교체하세요.
- 필요시 replicas, 리소스, 네트워크 설정 등은 YAML에서 조정하세요.

## 8. Log Analytics Workspace에서 컨테이너 로그 조회

이 단계에서는 AKS 클러스터에 배포된 컨테이너의 로그를 Log Analytics Workspace를 통해 조회하는 방법을 안내합니다.

- AKS 클러스터 생성 시 Log Analytics Workspace를 연동하면, Azure Portal에서 컨테이너의 로그와 모니터링 정보를 쉽게 확인할 수 있습니다.
- **Azure Portal에서 AKS 클러스터 > 모니터링 > 로그(Logs) 메뉴로 이동**하여 로그를 조회할 수 있습니다.
- 로그(Logs) 메뉴에서 Kusto 쿼리를 활용해 특정 컨테이너, 파드, 시간대별 로그를 조회할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능이나, kubectl logs 명령어를 사용할 수 있습니다.
- 로그 분석을 통해 배포 후 애플리케이션의 상태, 오류, 성능 이슈 등을 신속하게 파악할 수 있습니다.
- 필요에 따라 PodName, Namespace, LogLevel 등 다양한 조건으로 필터링하여 원하는 정보를 추출할 수 있습니다.

In [ ]:
# 예시: 컨테이너 v2 로그 Kusto 쿼리
# ContainerLogV2
# | where ContainerName == "travel-planning-agent-app"
# | sort by TimeGenerated desc
# | take 100

- 필요에 따라 PodName, Namespace, LogLevel 등으로 필터링할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능을 활용하거나, kubectl logs 명령어도 사용할 수 있습니다.

## 참고 자료 및 공식 가이드

- [Azure Kubernetes Service(AKS) 공식 문서](https://learn.microsoft.com/azure/aks/)
- [Azure Container Registry(ACR) 공식 문서](https://learn.microsoft.com/azure/container-registry/)
- [Docker 공식 문서](https://docs.docker.com/)
- [Kubernetes 공식 문서](https://kubernetes.io/docs/)
- [Azure CLI 공식 문서](https://learn.microsoft.com/cli/azure/)

위 링크를 참고하여 각 기술의 상세 사용법과 추가 예제를 확인할 수 있습니다.